In [1]:
from pygalfitm import PyGalfitm
from pygalfitm.VOs import splus
from pygalfitm.auxiliars import string_times_x, get_dims, get_exptime, unpack_file
from pygalfitm.psf import make_psf

import splusdata

import os

In [2]:
name = "ari_test"

ra = 52.430082897775755	
dec = -33.557141932864596
cut_size = 200
bands = ["I", "R", "G"]

DATA_FOLDER = "data/"
OUTPUT_FOLDER = "outputs/"

In [3]:
conn = splusdata.connect("gustavo", "asdflkjh")

You have access to internal data


In [4]:
pyg = PyGalfitm() 

In [5]:
input_images = ""
psf_images = ""
filters = ""

for band in bands:
    try:
        conn.get_cut(ra, dec, 200, band, filepath=os.path.join(DATA_FOLDER, f'{name}_{band.lower()}.fits'))
    except Exception as e:
        print(e)
    
    try:
        unpack_file(os.path.join(DATA_FOLDER, f'{name}_{band.lower()}.fits.fz'))
    except Exception as e:
        print(e)
        print("Make sure you have fpack (cfitsio) in your system alias.")
    
    make_psf(os.path.join(DATA_FOLDER, f'{name}_{band.lower()}.fits'), outfile=os.path.join(DATA_FOLDER, f'psf_{name}_{band.lower()}.fits'))

    input_images += "," + os.path.join(DATA_FOLDER, f'{name}_{band.lower()}.fits')
    psf_images += "," + os.path.join(DATA_FOLDER, f'psf_{name}_{band.lower()}.fits')

    filters += "," + str(band).lower()

input_images = input_images[1:]
psf_images = psf_images[1:]
filters = filters[1:]

In [6]:
import pandas as pd

df = pd.read_csv("pygalfitm/VOs/splus_idr4_zps.csv")

In [7]:
from astropy.io.fits import getheader

header = getheader(os.path.join(DATA_FOLDER, name + "_r.fits"))
field = header['OBJECT']

In [8]:
zps = ""

for band in bands:
    zps += "," + str(df[df['Field'] == field][f'ZP_{band.lower().replace("f", "J0")}'].values[0])

zps = zps[1:]

In [9]:
zps

'23.344,23.535,23.49'

In [10]:
pyg.set_base({
    "A": input_images,
    "A1": filters,
    "B": os.path.join(OUTPUT_FOLDER, name + "ss.fits"),
    "C": "none",
    "D": psf_images,
    "A2": "7625,6231,4770",
    "H": f"1   {cut_size}  1   {cut_size}",
    "I": f"{cut_size} {cut_size}",
    "J": zps,
    "K": "0.55 0.55"
})

In [11]:
pyg.print_base()

A) data/ari_test_i.fits,data/ari_test_r.fits,data/ari_test_g.fits # Input data image (FITS file)
A1) i,r,g                            # Nick names (band labels) 
A2) 7625,6231,4770                   # Effective wavelenghts
B) outputs/ari_testss.fits          # Output data image block
C) none                             # Sigma image name (made from data if blank or 'none')
D) data/psf_ari_test_i.fits,data/psf_ari_test_r.fits,data/psf_ari_test_g.fits # Input PSF image and (optional) diffusion kernel
E) 1                                # PSF fine sampling factor relative to data 
F) none                             # Bad pixel mask (FITS image or ASCII coord list)
G) none                             # File with parameter constraints (ASCII file) 
H) 1   200  1   200                 # Image region to fit (xmin xmax ymin ymax)
I) 200 200                          # Size of the convolution box (x y)
J) 23.344,23.535,23.49              # Magnitude photometric zeropoint
K) 0.55 0.55           

In [12]:
axis_ratios, effective_rs, position_angles, mags = splus.get_sersic_splus(conn, ra, dec, bands)

finished
finished
finished


In [13]:
pyg.set_component("sersic", "1", string_times_x(cut_size / 2, 3))
pyg.set_component("sersic", "2", string_times_x(cut_size / 2, 3))
pyg.set_component("sersic", "3", mags)
pyg.set_component("sersic", "4", effective_rs)
pyg.set_component("sersic", "5", string_times_x("4", 3))
pyg.set_component("sersic", "9", axis_ratios)
pyg.set_component("sersic", "10", position_angles)

pyg.print_component("sersic")

1) 100.0,100.0,100.0                   1    band       # Position x [pixel]
2) 100.0,100.0,100.0                   1    band       # Position y [pixel]
3) 14.022123,14.107952,14.922325       3    band       # Integrated magnitude
4) 25.440218,30.087149,24.733025       2    band       # R_e (effective radius) [pix]
5) 4,4,4                               2    band       # Sersic index n (de Vaucouleurs n=4)
9) 0.6433851,0.7046503,0.7636552       1    band       # Axis ratio (b/a)
10) 15.269562,23.058279,32.80171        1    band       # Position angle (PA) [deg: Up=0, Left=90]
Z) 0                                                   # Skip this model in output image? (yes=1, no=0)


In [20]:

pyg.active_components = ["sersic"]

In [21]:
pyg.write_feedme()

In [22]:
pyg.run()

b'\nGALFITM Version galfitm-1.4.4\n(based on GALFIT Version 3.0.5)\n\n\n#  Input menu file: galfit.feedme\n\n================================================================================\n# IMAGE and GALFIT CONTROL PARAMETERS\nA) data/ari_test_i.fits,data/ari_test_r.fits,data/ari_test_g.fits      # Input data image (FITS file)\nA1) i,r,g              # Band labels\nA2) 7625.00,6231.00,4770.00     # Band wavelengths\nB) outputs/ari_testss.fits      # Output data image block\nC) none,none,none 0.000   # Sigma image and min. sigma factor (made from data if blank or "none") \nD) data/psf_ari_test_i.fits,data/psf_ari_test_r.fits,data/psf_ari_test_g.fits #        # Input PSF image and (optional) diffusion kernel\nE) 1                   # PSF fine sampling factor relative to data \nF) none,none,none      # Bad pixel mask (FITS image or ASCII coord list)\nG) none                # File with parameter constraints (ASCII file) \nH) 1    200  1    200  # Image region to fit (xmin xmax ymin ymax

In [23]:
output = b'\nGALFITM Version galfitm-1.4.4\n(based on GALFIT Version 3.0.5)\n\n\n#  Input menu file: galfit.feedme\n\n================================================================================\n# IMAGE and GALFIT CONTROL PARAMETERS\nA) data/ari_test_i.fits,data/ari_test_r.fits,data/ari_test_g.fits      # Input data image (FITS file)\nA1) i,r,g              # Band labels\nA2) 7625.00,6231.00,4770.00     # Band wavelengths\nB) outputs/ari_testss.fits      # Output data image block\nC) none,none,none 0.000   # Sigma image and min. sigma factor (made from data if blank or "none") \nD) data/psf_ari_test_i.fits,data/psf_ari_test_r.fits,data/psf_ari_test_g.fits #        # Input PSF image and (optional) diffusion kernel\nE) 1                   # PSF fine sampling factor relative to data \nF) none,none,none      # Bad pixel mask (FITS image or ASCII coord list)\nG) none                # File with parameter constraints (ASCII file) \nH) 1    200  1    200  # Image region to fit (xmin xmax ymin ymax)\nI) 200    200          # Size of the convolution box (x y)\nJ) 23.344,23.535,23.490      # Magnitude photometric zeropoint \nK) 0.550  0.550        # Plate scale (dx dy)   [arcsec per pixel]\nO) regular             # Display type (regular, curses, both)\nP) 0                   # Choose: 0=optimize, 1=model, 2=imgblock, 3=subcomps\nU) 0 0.750000 25 4 40 0.000000 1.000000  # Non-parametric component settings\nV) 0 0 50 0.800000 0.500000 100000   # MultiNest options\nW) blank,input,model,residual      # Output options\n\n# INITIAL CHEBYSHEV FITTING PARAMETERS\n#\n#   For component type, the allowed functions are: \n#       sersic, expdisk, edgedisk, devauc, king, nuker, psf, \n#       gaussian, moffat, ferrer, corser, and sky. \n#  \n#   Hidden parameters will only appear when they\'re specified:\n#       Bn (n=integer, Bending Modes).\n#       C0 (diskyness/boxyness), \n#       Fn (n=integer, Azimuthal Fourier Modes).\n#       R0-R10 (coordinate rotation, for creating spiral structures).\n#       To, Ti, T0-T10 (truncation function).\n# \n# ------------------------------------------------------------------------------\n#   par)    par value(s)    fit toggle(s)    # parameter description \n# ------------------------------------------------------------------------------\n\n# Component number: 1\n 0) sersic                 #  Component type\n 1) 100.000,0,9.474e-15    1,0,0               cheb #      Position x\n 2) 100.000,0,9.474e-15    1,0,0               cheb #      Position y\n 3) 14.295,-0.450,0.177    1,1,1               cheb #  Integrated magnitude \n 4) 27.584,0.354,0    1,1,0               cheb #  R_e (effective radius)   [pix]\n 5) 4.000,4.441e-16,0    1,1,0               cheb #  Sersic index n (de Vaucouleurs n=4) \n 6) 0,0,0             0,0,0               cheb #     ----- \n 7) 0,0,0             0,0,0               cheb #     ----- \n 8) 0,0,0             0,0,0               cheb #     ----- \n 9) 0.705,-6.014e-02,-1.271e-03    1,0,0               cheb #  Axis ratio (b/a)  \n10) 23.650,-8.766,0.386    1,0,0               cheb #  Position angle (PA) [deg: Up=0, Left=90]\n Z) 0                      #  Skip this model in output image?  (yes=1, no=0)\n\n================================================================================\n\n# INITIAL BAND FITTING PARAMETERS\n#\n#   For component type, the allowed functions are: \n#       sersic, expdisk, edgedisk, devauc, king, nuker, psf, \n#       gaussian, moffat, ferrer, corser, and sky. \n#  \n#   Hidden parameters will only appear when they\'re specified:\n#       Bn (n=integer, Bending Modes).\n#       C0 (diskyness/boxyness), \n#       Fn (n=integer, Azimuthal Fourier Modes).\n#       R0-R10 (coordinate rotation, for creating spiral structures).\n#       To, Ti, T0-T10 (truncation function).\n# \n# ------------------------------------------------------------------------------\n#   par)    par value(s)    fit toggle(s)    # parameter description \n# ------------------------------------------------------------------------------\n\n# Component number: 1\n 0) sersic                 #  Component type\n 1) 100.000,100.000,100.000    1,0,0               band #      Position x\n 2) 100.000,100.000,100.000    1,0,0               band #      Position y\n 3) 14.022,14.108,14.922    1,1,1               band #  Integrated magnitude \n 4) 27.938,27.592,27.231    1,1,0               band #  R_e (effective radius)   [pix]\n 5) 4.000,4.000,4.000    1,1,0               band #  Sersic index n (de Vaucouleurs n=4) \n 6) 0,0,0             0,0,0               band #     ----- \n 7) 0,0,0             0,0,0               band #     ----- \n 8) 0,0,0             0,0,0               band #     ----- \n 9) 0.643,0.705,0.764    1,0,0               band #  Axis ratio (b/a)  \n10) 15.270,23.058,32.802    1,0,0               band #  Position angle (PA) [deg: Up=0, Left=90]\n Z) 0                      #  Skip this model in output image?  (yes=1, no=0)\n\n================================================================================\n\n\n\n-- No constraint file found.\n\n\n\n-- No CCD charge diffusion kernel found or applied.\n\n\n-- WARNING: No valid pixel mask images found.\n            Using blank pixel masks.\n\n-- No sigma image for band i.\n   Creating one using: GAIN=131.76, RDNOISE=0.00, NCOMBINE=3.0.\n-- No sigma image for band r.\n   Creating one using: GAIN=114.60, RDNOISE=0.00, NCOMBINE=3.0.\n-- No sigma image for band g.\n   Creating one using: GAIN=94.64, RDNOISE=0.00, NCOMBINE=3.0.\n\n-- WARNING: No sigma images found.\n            Calculating from data and supplied parameters.\n\n\n-- Estimating the sky mean and RMS to generate an internal sigma image.  This\n   estimate won\'t work right if the product ADUxGAIN has a unit other than \n   [electrons] (e.g. MJy/sr or e-/sec). To turn off this feature, start by:   \n                 galfit -noskyest <filename>  \n   For other options, do:  \n                 galfit -help  \n\ni: Est\'d sky mean = 0.11, RMS = 0.20 ADUs.  (Are these fairly reasonable?)\nApplying a minimum sigma of 0.000 times the input image\nr: Est\'d sky mean = 0.10, RMS = 0.17 ADUs.  (Are these fairly reasonable?)\nApplying a minimum sigma of 0.000 times the input image\ng: Est\'d sky mean = 0.06, RMS = 0.13 ADUs.  (Are these fairly reasonable?)\nApplying a minimum sigma of 0.000 times the input image\n\n================================================================================\nInitial parameters for 1 components:\n i  : sersic    : (  100.00,   100.00)   14.02     27.94    4.00    0.64    15.27\n r  : sersic    : (  100.00,   100.00)   14.11     27.59    4.00    0.70    23.06\n g  : sersic    : (  100.00,   100.00)   14.92     27.23    4.00    0.76    32.80\n================================================================================\n\nImproved chisq\n\nIteration : 1     Chi2nu: 2.04092e+00   dChi2/Chi2: -4.08e+21   alamda: 1e-03     \ndelchi2/sqrt(chi2nu): 7.00e+26    avghist: 0.00e+00  \n i  : sersic    : (  100.06,   100.11)   13.41     59.71    3.76    0.83    34.46\n r  : sersic    : (  100.06,   100.11)   13.67     53.45    3.27    0.90    42.25\n g  : sersic    : (  100.06,   100.11)   14.48     46.90    2.77    0.95    51.99\nCOUNTDOWN = 499 \n\nPA wrapped for comp 1 (objtype: sersic ): 112.33 to -67.67\nImproved chisq\n\nIteration : 2     Chi2nu: 1.66736e+00   dChi2/Chi2: -2.24e-01   alamda: 1e-04     \ndelchi2/sqrt(chi2nu): 3.47e+04    avghist: 3.47e+04  \n i  : sersic    : (  100.27,   100.44)   13.39     52.48    1.26    0.67   -76.05\n r  : sersic    : (  100.27,   100.44)   13.73     45.50    1.24    0.74   -68.27\n g  : sersic    : (  100.27,   100.44)   14.45     38.18    1.22    0.79   -58.52\nCOUNTDOWN = 498 \n\nPA wrapped for comp 1 (objtype: sersic ): -107.27 to 72.73\nOne or more parameters tried to exceed hard limits\nParameter(s) constrained by hard limits or user-specified constraint(s)\nImproved chisq\nconstr_alamda_factor: 1000.000\n\nIteration : 3     Chi2nu: 1.56415e+00   dChi2/Chi2: -6.60e-02   alamda: 1e-01     \ndelchi2/sqrt(chi2nu): 9.90e+03    avghist: 2.23e+04  \n i  : sersic    : (  101.28,   101.90)   13.52     38.24    1.61    0.67    64.35\n r  : sersic    : (  101.28,   101.90)   13.81     36.99    1.62    0.74    72.14\n g  : sersic    : (  101.28,   101.90)   14.44     35.69    1.63    0.79    81.89\nCOUNTDOWN = 497 \n\nImproved chisq\n\nIteration : 4     Chi2nu: 1.38557e+00   dChi2/Chi2: -1.29e-01   alamda: 1e-02     \ndelchi2/sqrt(chi2nu): 1.82e+04    avghist: 2.09e+04  \n i  : sersic    : (  101.15,   101.64)   13.61     42.00    1.36    0.63    56.85\n r  : sersic    : (  101.15,   101.64)   13.88     40.42    1.43    0.69    64.64\n g  : sersic    : (  101.15,   101.64)   14.50     38.77    1.50    0.75    74.39\nCOUNTDOWN = 496 \n\nImproved chisq\n\nIteration : 5     Chi2nu: 1.37588e+00   dChi2/Chi2: -7.04e-03   alamda: 1e-03     \ndelchi2/sqrt(chi2nu): 9.92e+02    avghist: 1.60e+04  \n i  : sersic    : (  101.04,   101.45)   13.60     45.15    1.34    0.57    56.89\n r  : sersic    : (  101.04,   101.45)   13.86     43.85    1.43    0.63    64.68\n g  : sersic    : (  101.04,   101.45)   14.46     42.48    1.54    0.69    74.42\nCOUNTDOWN = 495 \n\nImproved chisq\n\nIteration : 6     Chi2nu: 1.37546e+00   dChi2/Chi2: -3.07e-04   alamda: 1e-04     \ndelchi2/sqrt(chi2nu): 4.32e+01    avghist: 1.28e+04  \n i  : sersic    : (  101.01,   101.40)   13.59     46.05    1.34    0.56    56.64\n r  : sersic    : (  101.01,   101.40)   13.84     45.00    1.45    0.62    64.43\n g  : sersic    : (  101.01,   101.40)   14.44     43.89    1.56    0.68    74.17\nCOUNTDOWN = 494 \n\nImproved chisq\n\nIteration : 7     Chi2nu: 1.37545e+00   dChi2/Chi2: -9.92e-06   alamda: 1e-05     \ndelchi2/sqrt(chi2nu): 1.40e+00    avghist: 1.06e+04  \n i  : sersic    : (  101.00,   101.39)   13.59     46.18    1.35    0.56    56.62\n r  : sersic    : (  101.00,   101.39)   13.84     45.17    1.45    0.62    64.40\n g  : sersic    : (  101.00,   101.39)   14.44     44.11    1.57    0.68    74.15\nCOUNTDOWN = 20 \n\nImproved chisq\n\nIteration : 8     Chi2nu: 1.37545e+00   dChi2/Chi2: -9.09e-08   alamda: 1e-06     \ndelchi2/sqrt(chi2nu): 1.28e-02    avghist: 9.12e+03  \n i  : sersic    : (  101.00,   101.39)   13.59     46.19    1.35    0.56    56.61\n r  : sersic    : (  101.00,   101.39)   13.84     45.18    1.45    0.62    64.40\n g  : sersic    : (  101.00,   101.39)   14.43     44.13    1.57    0.68    74.15\nCOUNTDOWN = 19 \n\nImproved chisq\n\nIteration : 9     Chi2nu: 1.37545e+00   dChi2/Chi2: -7.19e-09   alamda: 1e-07     \ndelchi2/sqrt(chi2nu): 1.01e-03    avghist: 7.98e+03  \n i  : sersic    : (  101.00,   101.39)   13.59     46.19    1.35    0.56    56.61\n r  : sersic    : (  101.00,   101.39)   13.84     45.19    1.45    0.62    64.40\n g  : sersic    : (  101.00,   101.39)   14.43     44.13    1.57    0.68    74.14\nCOUNTDOWN = 18 \n\nImproved chisq\n\nIteration : 10    Chi2nu: 1.37545e+00   dChi2/Chi2: -1.07e-09   alamda: 1e-08     \ndelchi2/sqrt(chi2nu): 1.50e-04    avghist: 7.09e+03  \n i  : sersic    : (  101.00,   101.39)   13.59     46.19    1.35    0.56    56.61\n r  : sersic    : (  101.00,   101.39)   13.84     45.19    1.45    0.62    64.40\n g  : sersic    : (  101.00,   101.39)   14.43     44.13    1.57    0.68    74.14\nCOUNTDOWN = 17 \n\nImproved chisq\n\nIteration : 11    Chi2nu: 1.37545e+00   dChi2/Chi2: -1.86e-10   alamda: 1e-09     \ndelchi2/sqrt(chi2nu): 2.62e-05    avghist: 6.39e+03  \n i  : sersic    : (  101.00,   101.39)   13.59     46.19    1.35    0.56    56.61\n r  : sersic    : (  101.00,   101.39)   13.84     45.19    1.45    0.62    64.40\n g  : sersic    : (  101.00,   101.39)   14.43     44.13    1.57    0.68    74.14\nCOUNTDOWN = 16 \n\nImproved chisq\n\nIteration : 12    Chi2nu: 1.37545e+00   dChi2/Chi2: -3.48e-11   alamda: 1e-10     \ndelchi2/sqrt(chi2nu): 4.89e-06    avghist: 2.91e+03  \n i  : sersic    : (  101.00,   101.39)   13.59     46.19    1.35    0.56    56.61\n r  : sersic    : (  101.00,   101.39)   13.84     45.19    1.45    0.62    64.40\n g  : sersic    : (  101.00,   101.39)   14.43     44.13    1.57    0.68    74.14\nCOUNTDOWN = 15 \n\nImproved chisq\n\nIteration : 13    Chi2nu: 1.37545e+00   dChi2/Chi2: -6.34e-12   alamda: 1e-11     \ndelchi2/sqrt(chi2nu): 8.92e-07    avghist: 1.92e+03  \n i  : sersic    : (  101.00,   101.39)   13.59     46.19    1.35    0.56    56.61\n r  : sersic    : (  101.00,   101.39)   13.84     45.19    1.45    0.62    64.40\n g  : sersic    : (  101.00,   101.39)   14.43     44.13    1.57    0.68    74.14\nCOUNTDOWN = 14 \n\nImproved chisq\n\nIteration : 14    Chi2nu: 1.37545e+00   dChi2/Chi2: -1.20e-12   alamda: 1e-12     \ndelchi2/sqrt(chi2nu): 1.69e-07    avghist: 1.04e+02  \n i  : sersic    : (  101.00,   101.39)   13.59     46.19    1.35    0.56    56.61\n r  : sersic    : (  101.00,   101.39)   13.84     45.19    1.45    0.62    64.40\n g  : sersic    : (  101.00,   101.39)   14.43     44.13    1.57    0.68    74.14\nCOUNTDOWN = 13 \n\nImproved chisq\n\nIteration : 15    Chi2nu: 1.37545e+00   dChi2/Chi2: -2.09e-13   alamda: 1e-13     \ndelchi2/sqrt(chi2nu): 2.94e-08    avghist: 4.46e+00  \n i  : sersic    : (  101.00,   101.39)   13.59     46.19    1.35    0.56    56.61\n r  : sersic    : (  101.00,   101.39)   13.84     45.19    1.45    0.62    64.40\n g  : sersic    : (  101.00,   101.39)   14.43     44.13    1.57    0.68    74.14\nCOUNTDOWN = 12 \n\nImproved chisq\n\nIteration : 16    Chi2nu: 1.37545e+00   dChi2/Chi2: -6.00e-14   alamda: 1e-14     \ndelchi2/sqrt(chi2nu): 8.44e-09    avghist: 1.41e-01  \n i  : sersic    : (  101.00,   101.39)   13.59     46.19    1.35    0.56    56.61\n r  : sersic    : (  101.00,   101.39)   13.84     45.19    1.45    0.62    64.40\n g  : sersic    : (  101.00,   101.39)   14.43     44.13    1.57    0.68    74.14\nCOUNTDOWN = 11 \n\nWorse chisq, increasing alamda by factor of 10 -> next step smaller\n\nIteration : 17    Chi2nu: 1.37545e+00   dChi2/Chi2: 0.00e+00    alamda: 1e-13     \ndelchi2/sqrt(chi2nu): 0.00e+00    avghist: 1.40e-03  \n i  : sersic    : (  101.00,   101.39)   13.59     46.19    1.35    0.56    56.61\n r  : sersic    : (  101.00,   101.39)   13.84     45.19    1.45    0.62    64.40\n g  : sersic    : (  101.00,   101.39)   14.43     44.13    1.57    0.68    74.14\nCOUNTDOWN = 10 \n\nWorse chisq, increasing alamda by factor of 10 -> next step smaller\n\nIteration : 18    Chi2nu: 1.37545e+00   dChi2/Chi2: 0.00e+00    alamda: 1e-12     \ndelchi2/sqrt(chi2nu): 0.00e+00    avghist: 1.19e-04  \n i  : sersic    : (  101.00,   101.39)   13.59     46.19    1.35    0.56    56.61\n r  : sersic    : (  101.00,   101.39)   13.84     45.19    1.45    0.62    64.40\n g  : sersic    : (  101.00,   101.39)   14.43     44.13    1.57    0.68    74.14\nCOUNTDOWN = 9 \n\nWorse chisq, increasing alamda by factor of 10 -> next step smaller\n\nIteration : 19    Chi2nu: 1.37545e+00   dChi2/Chi2: 0.00e+00    alamda: 1e-11     \ndelchi2/sqrt(chi2nu): 0.00e+00    avghist: 1.83e-05  \n i  : sersic    : (  101.00,   101.39)   13.59     46.19    1.35    0.56    56.61\n r  : sersic    : (  101.00,   101.39)   13.84     45.19    1.45    0.62    64.40\n g  : sersic    : (  101.00,   101.39)   14.43     44.13    1.57    0.68    74.14\nCOUNTDOWN = 8 \n\nWorse chisq, increasing alamda by factor of 10 -> next step smaller\n\nIteration : 20    Chi2nu: 1.37545e+00   dChi2/Chi2: 0.00e+00    alamda: 1e-10     \ndelchi2/sqrt(chi2nu): 0.00e+00    avghist: 3.22e-06  \n i  : sersic    : (  101.00,   101.39)   13.59     46.19    1.35    0.56    56.61\n r  : sersic    : (  101.00,   101.39)   13.84     45.19    1.45    0.62    64.40\n g  : sersic    : (  101.00,   101.39)   14.43     44.13    1.57    0.68    74.14\nCOUNTDOWN = 7 \n\nWorse chisq, increasing alamda by factor of 10 -> next step smaller\n\nIteration : 21    Chi2nu: 1.37545e+00   dChi2/Chi2: 0.00e+00    alamda: 1e-09     \ndelchi2/sqrt(chi2nu): 0.00e+00    avghist: 5.99e-07  \n i  : sersic    : (  101.00,   101.39)   13.59     46.19    1.35    0.56    56.61\n r  : sersic    : (  101.00,   101.39)   13.84     45.19    1.45    0.62    64.40\n g  : sersic    : (  101.00,   101.39)   14.43     44.13    1.57    0.68    74.14\nCOUNTDOWN = 6 \n\nWorse chisq, increasing alamda by factor of 10 -> next step smaller\n\nIteration : 22    Chi2nu: 1.37545e+00   dChi2/Chi2: 0.00e+00    alamda: 1e-08     \ndelchi2/sqrt(chi2nu): 0.00e+00    avghist: 1.10e-07  \n i  : sersic    : (  101.00,   101.39)   13.59     46.19    1.35    0.56    56.61\n r  : sersic    : (  101.00,   101.39)   13.84     45.19    1.45    0.62    64.40\n g  : sersic    : (  101.00,   101.39)   14.43     44.13    1.57    0.68    74.14\nCOUNTDOWN = 5 \n\nWorse chisq, increasing alamda by factor of 10 -> next step smaller\n\nIteration : 23    Chi2nu: 1.37545e+00   dChi2/Chi2: 0.00e+00    alamda: 1e-07     \ndelchi2/sqrt(chi2nu): 0.00e+00    avghist: 2.07e-08  \n i  : sersic    : (  101.00,   101.39)   13.59     46.19    1.35    0.56    56.61\n r  : sersic    : (  101.00,   101.39)   13.84     45.19    1.45    0.62    64.40\n g  : sersic    : (  101.00,   101.39)   14.43     44.13    1.57    0.68    74.14\nCOUNTDOWN = 4 \n\nWorse chisq, increasing alamda by factor of 10 -> next step smaller\n\nIteration : 24    Chi2nu: 1.37545e+00   dChi2/Chi2: 0.00e+00    alamda: 1e-06     \ndelchi2/sqrt(chi2nu): 0.00e+00    avghist: 3.79e-09  \n i  : sersic    : (  101.00,   101.39)   13.59     46.19    1.35    0.56    56.61\n r  : sersic    : (  101.00,   101.39)   13.84     45.19    1.45    0.62    64.40\n g  : sersic    : (  101.00,   101.39)   14.43     44.13    1.57    0.68    74.14\nCOUNTDOWN = 3 \n\nWorse chisq, increasing alamda by factor of 10 -> next step smaller\n\nIteration : 25    Chi2nu: 1.37545e+00   dChi2/Chi2: 0.00e+00    alamda: 1e-05     \ndelchi2/sqrt(chi2nu): 0.00e+00    avghist: 8.44e-10  \n i  : sersic    : (  101.00,   101.39)   13.59     46.19    1.35    0.56    56.61\n r  : sersic    : (  101.00,   101.39)   13.84     45.19    1.45    0.62    64.40\n g  : sersic    : (  101.00,   101.39)   14.43     44.13    1.57    0.68    74.14\nCOUNTDOWN = 2 \n\nWorse chisq, increasing alamda by factor of 10 -> next step smaller\n\nIteration : 26    Chi2nu: 1.37545e+00   dChi2/Chi2: 0.00e+00    alamda: 1e-04     \ndelchi2/sqrt(chi2nu): 0.00e+00    avghist: 0.00e+00  \n i  : sersic    : (  101.00,   101.39)   13.59     46.19    1.35    0.56    56.61\n r  : sersic    : (  101.00,   101.39)   13.84     45.19    1.45    0.62    64.40\n g  : sersic    : (  101.00,   101.39)   14.43     44.13    1.57    0.68    74.14\nCOUNTDOWN = 1 \n\nWorse chisq, increasing alamda by factor of 10 -> next step smaller\n\nIteration : 27    Chi2nu: 1.37545e+00   dChi2/Chi2: 0.00e+00    alamda: 1e-03     \ndelchi2/sqrt(chi2nu): 0.00e+00    avghist: 0.00e+00  \n i  : sersic    : (  101.00,   101.39)   13.59     46.19    1.35    0.56    56.61\n r  : sersic    : (  101.00,   101.39)   13.84     45.19    1.45    0.62    64.40\n g  : sersic    : (  101.00,   101.39)   14.43     44.13    1.57    0.68    74.14\nCOUNTDOWN = 0 \n\n\n\nFit summary is now being saved into `fit.log\'.\n\n\x07'

In [25]:
print(output)


GALFITM Version galfitm-1.4.4
(based on GALFIT Version 3.0.5)


#  Input menu file: galfit.feedme

# IMAGE and GALFIT CONTROL PARAMETERS
A) data/ari_test_i.fits,data/ari_test_r.fits,data/ari_test_g.fits      # Input data image (FITS file)
A1) i,r,g              # Band labels
A2) 7625.00,6231.00,4770.00     # Band wavelengths
B) outputs/ari_testss.fits      # Output data image block
C) none,none,none 0.000   # Sigma image and min. sigma factor (made from data if blank or "none") 
D) data/psf_ari_test_i.fits,data/psf_ari_test_r.fits,data/psf_ari_test_g.fits #        # Input PSF image and (optional) diffusion kernel
E) 1                   # PSF fine sampling factor relative to data 
F) none,none,none      # Bad pixel mask (FITS image or ASCII coord list)
G) none                # File with parameter constraints (ASCII file) 
H) 1    200  1    200  # Image region to fit (xmin xmax ymin ymax)
I) 200    200          # Size of the convolution box (x y)
J) 23.344,23.535,23.490      # Magnitude